In [7]:
# 9/6/2024
# Preprocessing and Cleaning
import pandas as pd
import numpy as np
from datetime import datetime


def filter_by_time_format(df, num_rounds=3):
    """
    Filters the dataset to include only fights that match the specified number of rounds (default is 3).
    
    Parameters:
    - df (pd.DataFrame): The dataframe containing fight data.
    - num_rounds (int): The number of rounds to filter by. Default is 3 rounds.

    Returns:
    - pd.DataFrame: A filtered dataframe with only fights that match the specified round count.
    """
    first_char_series = pd.to_numeric(df['time_format'].astype(str).str[0], errors='coerce')
    df = df[first_char_series == num_rounds].copy()

    if num_rounds == 3:
        cols_to_drop = [col for col in df.columns if '4' in col or '5' in col]
        df = df.drop(columns=cols_to_drop)
        
    print(f"Columns filtered based on the number of rounds: {cols_to_drop}")
    print(f"DataFrame shape after time format filtering: {df.shape}\n")

    return df


def filter_by_early_stoppage(df, stop_round=2):
    """
    Filters the dataset to exclude fights that ended in the specified round or earlier, ensuring only longer fights are analyzed.
    
    Parameters:
    - df (pd.DataFrame): The dataframe containing fight data.
    - stop_round (int): The round by which the fight should not have ended. Default is 2 rounds.

    Returns:
    - pd.DataFrame: A filtered dataframe excluding early stoppages.
    """
    rounds_numeric = pd.to_numeric(df['round_of_victory'], errors='coerce')
    filtered_df = df[rounds_numeric > stop_round]
    
    print(f"Rows filtered by early stoppage successfully.")
    print(f"DataFrame shape after early stoppage filtering: {filtered_df.shape}\n")
    
    return filtered_df


def filter_by_date(df, start_date, end_date=None):
    """
    Filters the dataset based on a date range for fights. Only fights that occurred between the start and end dates are kept.
    
    Parameters:
    - df (pd.DataFrame): The dataframe containing fight data.
    - start_date (str): The start date in the format 'YYYY-MM-DD'.
    - end_date (str): The end date in the format 'YYYY-MM-DD'. Defaults to the current date if not provided.

    Returns:
    - pd.DataFrame: A filtered dataframe with fights within the specified date range.
    """
    df_copy = df.copy()
    df_copy['event_date'] = pd.to_datetime(df_copy['event_date'], format='%m-%d-%Y')

    if end_date is None:
        end_date = datetime.now().strftime('%Y-%m-%d')

    start_datetime = pd.to_datetime(start_date, format='%Y-%m-%d')
    end_datetime = pd.to_datetime(end_date, format='%Y-%m-%d')

    filtered_df = df_copy[(df_copy['event_date'] >= start_datetime) & (df_copy['event_date'] <= end_datetime)]

    print(f"Rows filtered by date successfully.")
    print(f"DataFrame shape after date filtering: {filtered_df.shape}\n")

    return filtered_df


def drop_NaN_values(df):
    """
    Removes rows containing missing values from the dataset and returns the cleaned dataset.
    
    Parameters:
    - df (pd.DataFrame): The dataframe containing fight data.

    Returns:
    - pd.DataFrame: A cleaned dataframe with no missing values.
    - pd.DataFrame: A dataframe with the rows that were dropped due to missing values.
    """
    mask = df.isna().any(axis=1)
    dropped_rows_df = df[mask]
    cleaned_df = df.dropna()
    
    print("Rows with NaN values have been dropped successfully.")
    print(f"DataFrame shape after dropping NaN values: {cleaned_df.shape}\n")
    
    return cleaned_df, dropped_rows_df


def filter_by_gender(df, gender):
    """
    Filters the dataset to include only fights from the specified gender division.
    
    Parameters:
    - df (pd.DataFrame): The dataframe containing fight data.
    - gender (str): The gender to filter by ('men' or 'women').

    Returns:
    - pd.DataFrame: A filtered dataframe with only fights from the specified gender division.
    """
    filtered_df = df[df['gender'].str.lower() == gender.lower()]
    
    print(f"Rows filtered by gender '{gender}' successfully.")
    print(f"DataFrame shape after gender filtering: {filtered_df.shape}\n")
    
    return filtered_df


def dob_to_age(dob_date, event_date):
    """
    Converts a fighter's date of birth into their age at the time of the fight, accounting for fractional years.
    
    Parameters:
    - dob_date (str): The date of birth of the fighter in the format 'MM-DD-YYYY'.
    - event_date (str): The event date in the format 'MM-DD-YYYY'.

    Returns:
    - float: The fighter's age as a fractional year at the time of the event.
    """
    if pd.isna(dob_date) or pd.isna(event_date):
        return None  # Handle missing values gracefully

    try:
        dob_date = pd.to_datetime(dob_date, format='%m-%d-%Y', errors='coerce')
        event_date = pd.to_datetime(event_date, format='%m-%d-%Y', errors='coerce')
    except Exception as e:
        print(f"Error converting dates: {e}")
        return None

    if pd.isna(dob_date) or pd.isna(event_date):
        return None  # Handle any failed conversions gracefully

    age = event_date.year - dob_date.year - ((event_date.month, event_date.day) < (dob_date.month, dob_date.day))

    try:
        next_birthday = datetime(event_date.year, dob_date.month, dob_date.day)
    except ValueError:
        if dob_date.month == 2 and dob_date.day == 29:
            next_birthday = datetime(event_date.year, 3, 1)
        else:
            raise

    if event_date > next_birthday:
        try:
            next_birthday = datetime(event_date.year + 1, dob_date.month, dob_date.day)
        except ValueError:
            next_birthday = datetime(event_date.year + 1, 3, 1)

    try:
        last_birthday = next_birthday.replace(year=next_birthday.year - 1)
    except ValueError:
        last_birthday = datetime(next_birthday.year - 1, 3, 1)

    days_since_last_birthday = (event_date - last_birthday).days
    days_in_year = (next_birthday - last_birthday).days
    fractional_year = days_since_last_birthday / days_in_year

    return round(age + fractional_year, 1)


def control_time_to_minutes(control_time):
    """
    Converts control time (in the format 'MM:SS') into total minutes.
    
    Parameters:
    - control_time (str): The control time in the format 'MM:SS'.

    Returns:
    - float: The control time converted into total minutes.
    """
    if pd.isna(control_time):
        return None  # Handle missing values gracefully

    try:
        minutes, seconds = map(int, control_time.split(':'))
        total_minutes = minutes + seconds / 60
        return round(total_minutes, 2)
    except Exception as e:
        print(f"Error converting time: {e}")
        return None

    
def attempted_landed_split(df, column_name):
    """
    Splits columns in the format 'X of Y' (e.g., strikes landed out of attempts) into two separate columns for 'landed' and 'attempted'.
    
    Parameters:
    - df (pd.DataFrame): The dataframe containing fight data.
    - column_name (str): The name of the column to split (e.g., 'sig_strikes').

    Returns:
    - None: Modifies the dataframe in place by adding new columns for 'landed' and 'attempted'.
    """
    if column_name in df.columns:
        df[column_name] = df[column_name].astype(str)  # Convert to string for safe splitting
        split_data = df[column_name].str.split(' of ', expand=True)

        if split_data.shape[1] == 2:
            df[column_name + '_landed'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
            df[column_name + '_attempted'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')
        else:
            df[column_name + '_landed'] = pd.NA
            df[column_name + '_attempted'] = pd.NA

        original_col_index = df.columns.get_loc(column_name)
        df.insert(original_col_index + 1, column_name + '_attempted', df.pop(column_name + '_attempted'))
        df.insert(original_col_index + 1, column_name + '_landed', df.pop(column_name + '_landed'))
        df.drop(column_name, axis=1, inplace=True)


def calculate_disparities(df, rounds=2):
    """
    Calculates the statistical disparities between fighters (e.g., strikes landed, control time) over the first two rounds.
    
    Parameters:
    - df (pd.DataFrame): The dataframe containing fight data.
    - rounds (int): The number of rounds to calculate disparities for. Default is 2 rounds.

    Returns:
    - pd.DataFrame: The dataframe with new columns representing the disparities for each fighter.
    """
    base_features = [
        'knockdowns', 'sig_strikes_landed', 'total_strikes_attempted', 'takedowns_landed',
        'takedowns_attempted', 'sub_attemps', 'control_time', 'body_strikes_landed',
        'leg_strikes_landed'
    ]
    
    for feature in base_features:
        disp_feature_a = f'{feature}_disp_a'
        disp_feature_b = f'{feature}_disp_b'

        # Initialize disparity columns
        df[disp_feature_a] = 0
        df[disp_feature_b] = 0

        for round_num in range(1, rounds + 1):
            a_feature = f'rnd_{round_num}_a_{feature}'
            b_feature = f'rnd_{round_num}_b_{feature}'

            if a_feature in df.columns and b_feature in df.columns:
                # Sum disparities over the first two rounds
                df[disp_feature_a] += df[a_feature] - df[b_feature]
                df[disp_feature_b] += -(df[a_feature] - df[b_feature])
    
    # Calculate height and age disparities
    df['height_disp_a'] = df['fighter_a_height'] - df['fighter_b_height']
    df['height_disp_b'] = -df['height_disp_a']
    
    df['age_disp_a'] = df['fighter_a_age'] - df['fighter_b_age']
    df['age_disp_b'] = -df['age_disp_a']

    return df


def split_fighter_statistics(df):
    """
    Splits the statistics for fighters into individual rows, where each row corresponds to one fighter's performance in a fight.
    This allows each fighter to be treated as an independent sample for machine learning purposes.
    
    Parameters:
    - df (pd.DataFrame): The dataframe containing fight data.

    Returns:
    - pd.DataFrame: A dataframe where each row represents one fighter's statistics for a fight, along with a fight ID.
    """
    columns_a = [col for col in df.columns if '_a_' in col or '_disp_a' in col]
    columns_b = [col for col in df.columns if '_b_' in col or '_disp_b' in col]

    data_a = df[columns_a].copy()
    data_b = df[columns_b].copy()

    data_a.columns = [col.replace('_a_', '_').replace('_disp_a', '_disp') for col in data_a.columns]
    data_b.columns = [col.replace('_b_', '_').replace('_disp_b', '_disp') for col in data_b.columns]

    data_a['outcome'] = 1  # Fighter A won
    data_b['outcome'] = 0  # Fighter B lost

    data_a.reset_index(drop=True, inplace=True)
    data_b.reset_index(drop=True, inplace=True)

    interleaved_data = pd.DataFrame(np.empty((len(data_a) + len(data_b), data_a.shape[1])), columns=data_a.columns)
    interleaved_data.iloc[::2, :] = data_a.values
    interleaved_data.iloc[1::2, :] = data_b.values

    split_df = pd.DataFrame(interleaved_data, columns=data_a.columns)
    
    # Assign fight_id starting at 1
    split_df['fight_id'] = np.repeat(range(1, len(data_a) + 1), 2)

    return split_df


def select_final_columns(df):
    """
    Selects the final columns to be used in the cleaned dataset, focusing on relevant statistics for each fight.
    
    Parameters:
    - df (pd.DataFrame): The dataframe containing fight data.

    Returns:
    - pd.DataFrame: A dataframe with only the selected columns that are relevant for analysis.
    """
    final_columns = [
        'fight_id', 'fighter_name', 'height_disp', 'age_disp', 'knockdowns_disp', 'sig_strikes_landed_disp', 
        'total_strikes_attempted_disp', 'takedowns_landed_disp', 'takedowns_attempted_disp', 
        'sub_attemps_disp', 'control_time_disp', 'body_strikes_landed_disp', 'leg_strikes_landed_disp', 
        'outcome'
    ]
    return df[final_columns]


# Main processing function
def process_and_clean_data(file_path):
    """
    The main function for preprocessing and cleaning fight data. It applies a series of filters, calculations, and transformations 
    to prepare the dataset for analysis.
    
    Parameters:
    - file_path (str): The path to the CSV file containing raw fight data.

    Returns:
    - None: Saves the cleaned dataset to a CSV file called 'final_cleaned_data.csv'.
    """
    df = pd.read_csv(file_path)
    df.replace(['--', ''], pd.NA, inplace=True)

    # Filter by rounds and other steps
    df = filter_by_time_format(df, num_rounds=3)
    df = filter_by_early_stoppage(df, stop_round=2)
    df = filter_by_date(df, start_date="2014-01-01")
    df, dropped_rows = drop_NaN_values(df)
    df = filter_by_gender(df, gender="men")

    # Convert DOB to age and control time to minutes
    dob_columns = ['fighter_a_dob', 'fighter_b_dob']
    for col in dob_columns:
        df[col.replace('_dob', '_age')] = df.apply(lambda row: dob_to_age(row[col], row['event_date']), axis=1)

    control_columns = [col for col in df.columns if 'control_time' in col]
    for col in control_columns:
        df[col] = df[col].apply(control_time_to_minutes)

    # Split columns with "Attempted of Landed" format
    columns_to_split = [
        'rnd_1_a_sig_strikes', 'rnd_1_b_sig_strikes', 'rnd_1_a_total_strikes', 'rnd_1_b_total_strikes',
        'rnd_1_a_takedowns', 'rnd_1_b_takedowns', 'rnd_1_a_body_strikes', 'rnd_1_b_body_strikes',
        'rnd_1_a_leg_strikes', 'rnd_1_b_leg_strikes', 'rnd_2_a_sig_strikes', 'rnd_2_b_sig_strikes',
        'rnd_2_a_total_strikes', 'rnd_2_b_total_strikes', 'rnd_2_a_takedowns', 'rnd_2_b_takedowns',
        'rnd_2_a_body_strikes', 'rnd_2_b_body_strikes', 'rnd_2_a_leg_strikes', 'rnd_2_b_leg_strikes'
    ]

    for col in columns_to_split:
        attempted_landed_split(df, col)

    # Calculate disparities for the first two rounds and sum them
    df = calculate_disparities(df, rounds=2)

    # Split fighter statistics into individual rows
    df = split_fighter_statistics(df)

    # Select final columns for the cleaned dataset
    df_final = select_final_columns(df)

    # Save the final dataset to a new CSV file
    df_final.to_csv('final_cleaned_data.csv', index=False)
    print("Final cleaned data has been saved as 'final_cleaned_data.csv'")

# Usage example
file_path = r"C:\Users\EditZ\event_masterlist (updated 09-06-2024).csv"
process_and_clean_data(file_path)


Columns filtered based on the number of rounds: ['rnd_4_a_knockdowns', 'rnd_4_a_sig_strikes', 'rnd_4_a_total_strikes', 'rnd_4_a_takedowns', 'rnd_4_a_sub_attemps', 'rnd_4_a_reversals', 'rnd_4_a_control_time', 'rnd_4_a_head_strikes', 'rnd_4_a_body_strikes', 'rnd_4_a_leg_strikes', 'rnd_4_a_distance_strikes', 'rnd_4_a_clinch_strikes', 'rnd_4_a_ground_strikes', 'rnd_4_b_knockdowns', 'rnd_4_b_sig_strikes', 'rnd_4_b_total_strikes', 'rnd_4_b_takedowns', 'rnd_4_b_sub_attemps', 'rnd_4_b_reversals', 'rnd_4_b_control_time', 'rnd_4_b_head_strikes', 'rnd_4_b_body_strikes', 'rnd_4_b_leg_strikes', 'rnd_4_b_distance_strikes', 'rnd_4_b_clinch_strikes', 'rnd_4_b_ground_strikes', 'rnd_5_a_knockdowns', 'rnd_5_a_sig_strikes', 'rnd_5_a_total_strikes', 'rnd_5_a_takedowns', 'rnd_5_a_sub_attemps', 'rnd_5_a_reversals', 'rnd_5_a_control_time', 'rnd_5_a_head_strikes', 'rnd_5_a_body_strikes', 'rnd_5_a_leg_strikes', 'rnd_5_a_distance_strikes', 'rnd_5_a_clinch_strikes', 'rnd_5_a_ground_strikes', 'rnd_5_b_knockdowns',